In [ ]:
import psycopg
from psycopg import sql
from psycopg.types.json import set_json_dumps, set_json_loads

import os
from dutchanalyzer.config import *
from dutchanalyzer.utilities.utils import *
from dutchanalyzer.utilities.json_utils import *
from dutchanalyzer.utilities.database_utils import *
from pathlib import Path
import ujson

In [102]:
from dutchanalyzer.utilities.util_vars import *

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [7]:
def get_connection():
    try:
        return psycopg.connect(f"dbname={os.getenv('cleaning_database')} user={os.getenv('database_username')} password={os.getenv('database_password')} host={os.getenv('database_host')} port={os.getenv('database_port')}"
        )
    except:
        return False

In [ ]:
with psycopg.connect(f"dbname={os.getenv('cleaning_database')} user={os.getenv('database_username')} password={os.getenv('database_password')} host={os.getenv('database_host')} port={os.getenv('database_port')}") as conn:

    # Open a cursor to perform database operations
    with conn.cursor() as cur:

        # Execute a command: this creates a new table
        if cur:
            
            cur.execute(""" 
                CREATE TABLE IF NOT EXISTS primary_parts_of_speech (
                pos_code TEXT PRIMARY KEY,
                dutch_name TEXT,   
                dutch_abbr TEXT,            
                english_name TEXT
                        )
            """)
            # 27
            cur.execute("""
            CREATE TABLE IF NOT EXISTS raw_entries (
                id bigserial PRIMARY KEY,
                word TEXT,
                pos TEXT REFERENCES primary_parts_of_speech(pos_code), 
                lang_code TEXT REFERENCES languages(lang_code),
                def_lang_code TEXT REFERENCES languages(lang_code),
                f_code TEXT, 
                processing_complete BOOL DEFAULT False,
                original_json JSONB, 
                added_on TIMESTAMPTZ,
                updated_at TIMESTAMPTZ
                    )
                """)
            #41
            cur.execute( 
                """ 
                CREATE TABLE IF NOT EXISTS processing_statuses (
                id bigserial PRIMARY KEY,
                raw_id BIGINT REFERENCES raw_entries(id),
                further_processing_status BOOL DEFAULT False,
                synonyms_status BOOL DEFAULT False,
                antoyms_status BOOL DEFAULT False,
                translations_status BOOL DEFAULT False,
                etymology_status BOOL DEFAULT False,
                links_status BOOL DEFAULT False,
                topics_status BOOL DEFAULT False,
                derived_status BOOL DEFAULT False,
                alt_status BOOL DEFAULT False,
                forms_status BOOL DEFAULT False,
                sounds_status BOOL DEFAULT False, 
                hyphenations_status BOOL DEFAULT False,
                inflection_templates_status BOOL DEFAULT False,
                categories_status BOOL DEFAULT False,
                hypernyms BOOL DEFAULT False,
                hyponyms BOOL DEFAULT False,
                meronyms BOOL DEFAULT False,
                toponyms BOOL DEFAULT False,
                related BOOL DEFAULT False,
                sense_status BOOL DEFAULT False,
                gloss_status BOOL DEFAULT False,
                short_gloss_status BOOL DEFAULT False, 
                sense_translations BOOL DEFAULT False, 
                sense_forms BOOL DEFAULT False, 
                sense_tags BOOL DEFAULT False, 
                sense_categories BOOL DEFAULT False, 
                sense_alt BOOL DEFAULT False, 
                sense_topics BOOL DEFAULT False, 
                sense_nym_status BOOL DEFAULT False 
                )
                """)
            

In [104]:
def alter_table(table_name, operation, column_name, rename_to='', db='cleaning_db'):
    if db == 'cleaning_db':
        with psycopg.connect(f"dbname={os.getenv('cleaning_database')} user={os.getenv('database_username')} password={os.getenv('database_password')} host={os.getenv('database_host')} port={os.getenv('database_port')}") as conn:
            with conn.cursor() as cur:
                if cur:
                    tble = sql.Identifier(table_name)
                    col = sql.Identifier(column_name)

                    if operation == 'rename_column':
                        if not rename_to:
                            print('requires new col name')
                            return False
                        query = t"""ALTER TABLE {table_name} 
                                RENAME COLUMN {column_name} to {rename_to};"""
                        cur.execute(query)

In [134]:
table_exists('primary_parts_of_speech')

True

In [ ]:
def create_primary_parts_of_speech_table(db='cleaning_db'):
    if db == 'cleaning_db':
        with psycopg.connect(f"dbname={os.getenv('cleaning_database')} user={os.getenv('database_username')} password={os.getenv('database_password')} host={os.getenv('database_host')} port={os.getenv('database_port')}") as conn:
            with conn.cursor() as cur:
                if cur:
                    
                    cur.execute(""" 
                                CREATE TABLE IF NOT EXISTS primary_parts_of_speech (
                                    pos_code TEXT PRIMARY KEY,
                                    nl_name TEXT,   
                                    nl_abbr TEXT,            
                                    en_name TEXT
                                    )
                                """)
                    for pos_code, vals in POS_INFO.items():
                        lang = vals['standard']
                        dutch_name = vals["dutch_name"]
                        english_name = vals['en_name']
                        cur.execute(t"INSERT INTO languages (lang_code, lang, dutch_name, english_name) VALUES ({lang_code}, {lang}, {dutch_name}, {english_name})")
                    

## Utilities

In [ ]:
def get_all_items_from_table(table, db='cleaning_db'):
    if db == 'cleaning_db':
        with psycopg.connect(f"dbname={os.getenv('cleaning_database')} user={os.getenv('database_username')} password={os.getenv('database_password')} host={os.getenv('database_host')} port={os.getenv('database_port')}") as conn:

            with conn.cursor() as cur:
                if cur:
                    items = cur.execute(t"SELECT * FROM  {table:i}")
                    return items.fetchall()

In [ ]:
get_all_items_from_table('languages')

[('en', 'english', 'engels', 'english'),
 ('eng', 'english', 'engels', 'english'),
 ('nl', 'dutch', 'nederlands', 'dutch'),
 ('nld', 'dutch', 'nederlands', 'dutch'),
 ('simple', 'simple_english', 'eenvoudig engels', 'simple english'),
 ('ang', 'old_english', 'oudengels', 'old english'),
 ('enm', 'middle_english', 'middelengels', 'middle english'),
 ('dum', 'middle_dutch', 'middelnederlands', 'middle dutch'),
 ('odt', 'old_dutch', 'oudnederlands', 'old dutch'),
 ('nds', 'low_saxon', 'nedersaksisch', 'low german / low saxon'),
 ('nds-nl', 'dutch_low_saxon', 'nedersaksisch (nederland)', 'dutch low saxon'),
 ('lim', 'limburgish', 'limburgs', 'limburgish'),
 ('osx', 'pre_dutch', 'oudsaksisch', 'pre_dutch')]

In [93]:
def get_table_col_schemas_in_db(db='cleaning_db'):
   if db == 'cleaning_db':
        with psycopg.connect(f"dbname={os.getenv('cleaning_database')} user={os.getenv('database_username')} password={os.getenv('database_password')} host={os.getenv('database_host')} port={os.getenv('database_port')}") as conn:
            with conn.cursor() as cur:
                if cur:
                    db_schema = {}

                    rows = cur.execute("""
                        SELECT 
                            table_schema,
                            table_name,
                            column_name,
                            data_type
                        FROM information_schema.columns
                        WHERE table_schema = 'public'
                        ORDER BY table_schema, table_name, ordinal_position;
                        """)
              
                    for schema, table, column, dtype in rows:
                        if table not in db_schema:
                            db_schema[table] = {column:dtype}
                        else:
                            db_schema[table][column] = dtype
        
                    return db_schema

In [100]:
get_table_col_schemas_in_db()

{'languages': {'lang_code': 'text',
  'lang': 'text',
  'dutch_name': 'text',
  'english_name': 'text'},
 'primary_parts_of_speech': {'pos_code': 'text',
  'dutch_name': 'text',
  'dutch_abbr': 'text',
  'english_name': 'text'},
 'processing_statuses': {'id': 'bigint',
  'raw_id': 'bigint',
  'further_processing_status': 'boolean',
  'synonyms_status': 'boolean',
  'antoyms_status': 'boolean',
  'translations_status': 'boolean',
  'etymology_status': 'boolean',
  'links_status': 'boolean',
  'topics_status': 'boolean',
  'derived_status': 'boolean',
  'alt_status': 'boolean',
  'forms_status': 'boolean',
  'sounds_status': 'boolean',
  'hyphenations_status': 'boolean',
  'inflection_templates_status': 'boolean',
  'categories_status': 'boolean',
  'hypernyms': 'boolean',
  'hyponyms': 'boolean',
  'meronyms': 'boolean',
  'toponyms': 'boolean',
  'related': 'boolean',
  'sense_status': 'boolean',
  'gloss_status': 'boolean',
  'short_gloss_status': 'boolean',
  'sense_translations': 'b

## Create Tables

In [ ]:
## Entries and Senses

# cur.execute(""" 
#                 CREATE TABLE IF NOT EXISTS sense_table (
#                 id bigserial PRIMARY KEY,
#                 wiki_entry_id NOT NULL REFERENCES wiktionary_entry(id),
                        
#                 original_json JSONB, 
#                 )
#             """
#             )

#             cur.execute(""" 
#                 CREATE TABLE IF NOT EXISTS gloss_table (
#                 id bigserial PRIMARY KEY,
#                 sense_id NOT NULL REFERENCES sense_table(id),
#                 gloss TEXT,        
#                 """)